# Performance Comparison: Text-Fabric vs Context-Fabric

This notebook compares loading performance and memory consumption between:
- **Text-Fabric (TF)**: The original implementation using pickle/gzip caching
- **Context-Fabric (CF)**: The new implementation using memory-mapped numpy arrays

We test against the BHSA (Biblia Hebraica Stuttgartensia Amstelodamensis) corpus.

In [1]:
import os
import sys
import gc
import time
import shutil
import psutil
from pathlib import Path

# Add Context-Fabric packages to path
sys.path.insert(0, str(Path.cwd().parent / 'packages'))

# Source paths
BHSA_SOURCE = '/Users/cody/github/etcbc/bhsa/tf/2021'
TFX_CACHE = Path(BHSA_SOURCE) / '.tf'
CFM_CACHE = Path(BHSA_SOURCE) / '.cfm'

print(f"BHSA Source: {BHSA_SOURCE}")
print(f"TFX Cache: {TFX_CACHE}")
print(f"CFM Cache: {CFM_CACHE}")

BHSA Source: /Users/cody/github/etcbc/bhsa/tf/2021
TFX Cache: /Users/cody/github/etcbc/bhsa/tf/2021/.tf
CFM Cache: /Users/cody/github/etcbc/bhsa/tf/2021/.cfm


## Utility Functions

In [2]:
def get_memory_usage_mb():
    """Get current process memory usage in MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def get_cache_size_mb(cache_path):
    """Get total size of cache directory in MB."""
    if not cache_path.exists():
        return 0
    total = 0
    for p in cache_path.rglob('*'):
        if p.is_file():
            total += p.stat().st_size
    return total / 1024 / 1024

def clear_caches():
    """Remove both TFX and CFM caches."""
    for cache in [TFX_CACHE, CFM_CACHE]:
        if cache.exists():
            shutil.rmtree(cache)
            print(f"Removed: {cache}")
        else:
            print(f"Not found: {cache}")

class TimingResult:
    """Store timing and memory results."""
    def __init__(self, name):
        self.name = name
        self.compile_time = 0
        self.load_time = 0
        self.memory_before = 0
        self.memory_after = 0
        self.cache_size = 0
    
    @property
    def memory_used(self):
        return self.memory_after - self.memory_before
    
    def __repr__(self):
        return (f"{self.name}:\n"
                f"  Compile time: {self.compile_time:.2f}s\n"
                f"  Load time:    {self.load_time:.2f}s\n"
                f"  Memory used:  {self.memory_used:.1f} MB\n"
                f"  Cache size:   {self.cache_size:.1f} MB")

## Step 1: Clear All Caches

Remove any existing `.tf` (Text-Fabric) and `.cfm` (Context-Fabric) caches to ensure fresh compilation.

In [3]:
# Clear all caches
clear_caches()

# Force garbage collection
gc.collect()

print("\nCaches cleared.")

Removed: /Users/cody/github/etcbc/bhsa/tf/2021/.tf


Removed: /Users/cody/github/etcbc/bhsa/tf/2021/.cfm

Caches cleared.


## Step 2: Text-Fabric Performance Test

Test the original Text-Fabric implementation using the low-level Fabric class.

In [4]:
from tf.fabric import Fabric as TFFabric

tf_result = TimingResult("Text-Fabric")

# Measure memory before
gc.collect()
tf_result.memory_before = get_memory_usage_mb()
print(f"Memory before: {tf_result.memory_before:.1f} MB")

# Time the initial load (compilation)
print("\nLoading Text-Fabric (first load - compiles cache)...")
start = time.perf_counter()
tf_TF = TFFabric(locations=BHSA_SOURCE, silent='deep')
tf_api = tf_TF.loadAll(silent='deep')  # loadAll loads all features
tf_result.compile_time = time.perf_counter() - start
print(f"Compile time: {tf_result.compile_time:.2f}s")

# Measure memory after
tf_result.memory_after = get_memory_usage_mb()
print(f"Memory after: {tf_result.memory_after:.1f} MB")
print(f"Memory used: {tf_result.memory_used:.1f} MB")

# Check cache size
tf_result.cache_size = get_cache_size_mb(TFX_CACHE)
print(f"Cache size: {tf_result.cache_size:.1f} MB")

Memory before: 72.4 MB

Loading Text-Fabric (first load - compiles cache)...


Compile time: 61.46s
Memory after: 4905.3 MB
Memory used: 4832.9 MB
Cache size: 137.9 MB


In [5]:
# Verify Text-Fabric loaded correctly
print("Verification - Checking F.otype:")
print(f"Max slot: {tf_api.F.otype.maxSlot}")
print(f"Max node: {tf_api.F.otype.maxNode}")
print(f"Node 1 type: {tf_api.F.otype.v(1)}")
print(f"First word: {tf_api.F.g_word_utf8.v(1)}")

Verification - Checking F.otype:
Max slot: 426590
Max node: 1446831
Node 1 type: word
First word: בְּ


In [6]:
# Clean up Text-Fabric objects to free memory
del tf_TF, tf_api
gc.collect()
print("Text-Fabric objects cleaned up.")

Text-Fabric objects cleaned up.


In [7]:
# Test reload time (from cache)
print("Reloading Text-Fabric (from cache)...")
gc.collect()

start = time.perf_counter()
tf_TF2 = TFFabric(locations=BHSA_SOURCE, silent='deep')
tf_api2 = tf_TF2.loadAll(silent='deep')
tf_result.load_time = time.perf_counter() - start
print(f"Load time (from cache): {tf_result.load_time:.2f}s")

# Clean up
del tf_TF2, tf_api2
gc.collect()

Reloading Text-Fabric (from cache)...


Load time (from cache): 7.30s


3984883

## Step 3: Context-Fabric Performance Test

Test the new Context-Fabric implementation with memory-mapped numpy arrays.

In [8]:
# Clear CFM cache (TFX cache will remain for fair comparison)
if CFM_CACHE.exists():
    shutil.rmtree(CFM_CACHE)
    print(f"Removed: {CFM_CACHE}")

gc.collect()
print("Ready for Context-Fabric test.")

Ready for Context-Fabric test.


In [9]:
from cfabric.core.fabric import Fabric as CFFabric

cf_result = TimingResult("Context-Fabric")

# Measure memory before
gc.collect()
cf_result.memory_before = get_memory_usage_mb()
print(f"Memory before: {cf_result.memory_before:.1f} MB")

# Time the initial load (compilation)
print("\nLoading Context-Fabric (first load - compiles cache)...")
start = time.perf_counter()
cf_TF = CFFabric(locations=BHSA_SOURCE, silent='deep')
cf_api = cf_TF.load('')  # Load all features
cf_result.compile_time = time.perf_counter() - start
print(f"Compile time: {cf_result.compile_time:.2f}s")

# Measure memory after
cf_result.memory_after = get_memory_usage_mb()
print(f"Memory after: {cf_result.memory_after:.1f} MB")
print(f"Memory used: {cf_result.memory_used:.1f} MB")

# Check cache size
cf_result.cache_size = get_cache_size_mb(CFM_CACHE)
print(f"Cache size: {cf_result.cache_size:.1f} MB")

Memory before: 1535.2 MB

Loading Context-Fabric (first load - compiles cache)...


   |     0.19s T otype                from ~/github/etcbc/bhsa/tf/2021


   |     3.39s T oslots               from ~/github/etcbc/bhsa/tf/2021


   |     0.00s T book@pt              from ~/github/etcbc/bhsa/tf/2021


   |     0.00s T book@en              from ~/github/etcbc/bhsa/tf/2021


...

 1m 28s   Compiling mother (edge, values=False)...


 1m 31s   Compiling omap@c-2021 (edge, values=True)...


 1m 33s Compilation complete


Compile time: 94.75s
Memory after: 1595.8 MB
Memory used: 60.6 MB


Cache size: 859.2 MB


In [10]:
# Verify Context-Fabric loaded correctly
print("Verification - Checking F.otype:")
print(f"Max slot: {cf_api.F.otype.maxSlot}")
print(f"Max node: {cf_api.F.otype.maxNode}")
print(f"Node 1 type: {cf_api.F.otype.v(1)}")
print(f"First word: {cf_api.F.g_word_utf8.v(1)}")

Verification - Checking F.otype:
Max slot: 426590
Max node: 1446831
Node 1 type: word
First word: בְּ


In [11]:
# Clean up Context-Fabric objects
del cf_TF, cf_api
gc.collect()
print("Context-Fabric objects cleaned up.")

Context-Fabric objects cleaned up.


In [12]:
# Test reload time (from cache)
print("Reloading Context-Fabric (from cache)...")
gc.collect()

start = time.perf_counter()
cf_TF2 = CFFabric(locations=BHSA_SOURCE, silent='deep')
cf_api2 = cf_TF2.load('')
cf_result.load_time = time.perf_counter() - start
print(f"Load time (from cache): {cf_result.load_time:.2f}s")

# Clean up
del cf_TF2, cf_api2
gc.collect()

Reloading Context-Fabric (from cache)...
  0.00s Loading from ~/github/etcbc/bhsa/tf/2021/.cfm/1


  2.53s All features loaded from .cfm format


Load time (from cache): 2.53s


997

## Step 4: Results Comparison

In [13]:
print("=" * 60)
print("PERFORMANCE COMPARISON RESULTS")
print("=" * 60)
print()
print(tf_result)
print()
print(cf_result)
print()
print("=" * 60)
print("SPEEDUP ANALYSIS")
print("=" * 60)

compile_speedup = tf_result.compile_time / cf_result.compile_time if cf_result.compile_time > 0 else 0
load_speedup = tf_result.load_time / cf_result.load_time if cf_result.load_time > 0 else 0
memory_ratio = cf_result.memory_used / tf_result.memory_used if tf_result.memory_used > 0 else 0
cache_ratio = cf_result.cache_size / tf_result.cache_size if tf_result.cache_size > 0 else 0

print(f"\nCompile speedup:     {compile_speedup:.2f}x {'faster' if compile_speedup > 1 else 'slower'}")
print(f"Load speedup:        {load_speedup:.2f}x {'faster' if load_speedup > 1 else 'slower'}")
print(f"Memory reduction:    {(1 - memory_ratio) * 100:.1f}%" if memory_ratio < 1 else f"Memory increase: {(memory_ratio - 1) * 100:.1f}%")
print(f"Cache size ratio:    {cache_ratio:.2f}x" + (" smaller" if cache_ratio < 1 else " larger"))

PERFORMANCE COMPARISON RESULTS

Text-Fabric:
  Compile time: 61.46s
  Load time:    7.30s
  Memory used:  4832.9 MB
  Cache size:   137.9 MB

Context-Fabric:
  Compile time: 94.75s
  Load time:    2.53s
  Memory used:  60.6 MB
  Cache size:   859.2 MB

SPEEDUP ANALYSIS

Compile speedup:     0.65x slower
Load speedup:        2.89x faster
Memory reduction:    98.7%
Cache size ratio:    6.23x larger


In [14]:
# Create a summary table
print("\n" + "=" * 70)
print(f"{'Metric':<20} {'Text-Fabric':<15} {'Context-Fabric':<15} {'Improvement':<15}")
print("=" * 70)
print(f"{'Compile Time (s)':<20} {tf_result.compile_time:<15.2f} {cf_result.compile_time:<15.2f} {compile_speedup:.2f}x")
print(f"{'Load Time (s)':<20} {tf_result.load_time:<15.2f} {cf_result.load_time:<15.2f} {load_speedup:.2f}x")
print(f"{'Memory Used (MB)':<20} {tf_result.memory_used:<15.1f} {cf_result.memory_used:<15.1f} {(1 - memory_ratio) * 100:.1f}%")
print(f"{'Cache Size (MB)':<20} {tf_result.cache_size:<15.1f} {cf_result.cache_size:<15.1f} {(1 - cache_ratio) * 100:.1f}%")
print("=" * 70)


Metric               Text-Fabric     Context-Fabric  Improvement    
Compile Time (s)     61.46           94.75           0.65x
Load Time (s)        7.30            2.53            2.89x
Memory Used (MB)     4832.9          60.6            98.7%
Cache Size (MB)      137.9           859.2           -523.0%


## Notes

- **Compile time**: Time to parse `.tf` files and build the cache (first load)
- **Load time**: Time to load from cache (subsequent loads)
- **Memory used**: Increase in process memory after loading
- **Cache size**: Total size of cache files on disk

Context-Fabric uses memory-mapped numpy arrays, which should show:
- Lower memory footprint (data stays on disk, mapped into virtual memory)
- Faster load times (no deserialization needed)
- Potentially different cache sizes (binary arrays vs compressed pickle)